2020年までの各情報（特徴量・味）

cup of excellence

In [1]:
import re
import time
from typing import Iterator
import requests
import lxml
import lxml.html
import cssselect
from pymongo import MongoClient

import pandas as pd

In [10]:
#クローラー

#各年directoryURLのリスト
def main():
    
    client = MongoClient('localhost', 27017)  # ローカルホストのMongoDBに接続する。
    collection = client.scraping.coe_results_2020  # scrapingデータベースのcoeコレクションを得る。
    collection.create_index('key',unique=True) # データを一意に識別するキーを格納するkeyフィールドにユニークなインデックスを作成する。
    
    headers = requests.utils.default_headers()
    headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.2 Safari/605.1.15'
    session = requests.Session()
    
    #2016~2020
    response1 = session.get('https://cupofexcellence.org/farm-directory/')
    html1 = lxml.html.fromstring(response1.text)
    #1999~2015
    response2 = session.get('https://cupofexcellence.org/directory/wpbdp_category/0000-previous-years/')
    html2 = lxml.html.fromstring(response2.text)
    #全yearリスト
    urls_year = []
    for a in html1.cssselect('a.category-label'):
        urls_year.append(a.get('href'))
    for a in html2.cssselect('a.category-label'):
        urls_year.append(a.get('href'))
    urls_year.remove('https://cupofexcellence.org/directory/wpbdp_category/0000-previous-years/')
    
    for url_year in urls_year: #全てのyearが格納されたリストから各年ごとに処理する
        time.sleep(2)
        response_year = session.get(url_year)  # あるYearの一覧htmlを取得する。
        urls_year_country = scrape_list_page(response_year)  # 全てのYear>>CountryページURLを得る。
        
        for url_year_country in urls_year_country:
            time.sleep(2)
            key_country_year = extract_key(url_year_country)  # URLからキーを取得する。（Country-Year）
            response_year_country = session.get(url_year_country)  # Year>>Countryのページを取得する。
            urls_score = scrape_result_page(response_year_country) # Year>>Country>>Rankのページ

            for url_score in urls_score:
                print(url_score)
                key_score = extract_key(url_score) #結果URLからキーを取得する。（Score）
                key = key_country_year + '-' + key_score #最終キー（country-year-score）
                coe_result = collection.find_one({'key': key})  # MongoDBからkeyに該当するデータを探す。
                if not coe_result:  # MongoDBに存在しない場合、詳細ページをクロールする。
                    time.sleep(2)
                    response_score = session.get(url_score)  # 詳細ページを取得する。
                    result = scrape_detail_page(response_score, key)
                    result = {k.replace('.',''): v for k, v in result.items() if type(k) != int}
                    collection.insert_one(result)  # 表の情報をMongoDBのcoeコレクションに保存する。
                    
#スクレイピング
def scrape_list_page(response: requests.Response) -> list:
    """
    一覧ページのResponseから詳細ページのURLを抜き出すジェネレーター関数。
    """
    html = lxml.html.fromstring(response.text)
    html.make_links_absolute(response.url)
    urls_year_country=[]
    
    for a in html.cssselect('a.wpbdp-category-icon-link[href^="https://"]'):
        urls_year_country.append(a.get('href'))
    return urls_year_country
        
def scrape_result_page(response: requests.Response) -> list:
    """
    YearCountryページのResponseから表とNextページのURLを抜き出すジェネレーター関数。Nextならもっかい。
    """
    session = requests.Session()
    urls_score = []

    def get_result(response):
        html = lxml.html.fromstring(response.text)
        html.make_links_absolute(response.url)
        for a_url_score in html.cssselect('a:contains("View")'): #各rank結果が掲載されているView先リンク取得
            urls_score.append(a_url_score.get('href'))
        else:
            a_url_next = html.cssselect('a:contains("Next")')
            for a in a_url_next:
                if not a_url_next:
                    break
                else:#Nextはgetしてからもう一回関数回す。
                    get_result(session.get(a.get('href')))

    get_result(response)
    return urls_score

def scrape_detail_page(response: requests.Response, key) -> dict:
    """
    詳細ページのResponseから表の情報をdictで取得する。
    """
    table = pd.read_html(response.content)
    data = {}
    #key, urlの追加
    #オークションに関するカップ情報の辞書を作成
    data.update(zip(table[-1][0], table[-1][1]))
    data.update([('key', key), ('url', response.url)])
    return data  # dictを返す。
    

def extract_key(url: str) -> str:
    """
    URLからキー（URLの末尾のISBN）を抜き出す。
    """
    m = re.search(r'/([^/]+)/$', url)  # 最後の/から文字列末尾までを正規表現で取得。
    return m.group(1)

if __name__ == '__main__':
    main()

https://cupofexcellence.org/directory/90-4/
https://cupofexcellence.org/directory/90-3/
https://cupofexcellence.org/directory/89-58-8/
https://cupofexcellence.org/directory/88-96-4/
https://cupofexcellence.org/directory/88-92-5/
https://cupofexcellence.org/directory/88-88-7/
https://cupofexcellence.org/directory/88-77-5/
https://cupofexcellence.org/directory/88-77-4/
https://cupofexcellence.org/directory/88-73-5/
https://cupofexcellence.org/directory/88-58-7/
https://cupofexcellence.org/directory/88-58-8/
https://cupofexcellence.org/directory/88-54-3/
https://cupofexcellence.org/directory/88-38-8/
https://cupofexcellence.org/directory/88-19-5/
https://cupofexcellence.org/directory/87-81-7/
https://cupofexcellence.org/directory/87-81-6/
https://cupofexcellence.org/directory/87-65-4/
https://cupofexcellence.org/directory/87-54-5/
https://cupofexcellence.org/directory/87-54-6/
https://cupofexcellence.org/directory/87-35-5/
https://cupofexcellence.org/directory/87-27-12/
https://cupofexcel

https://cupofexcellence.org/directory/85-36-11/
https://cupofexcellence.org/directory/85-27-2/
https://cupofexcellence.org/directory/90-27-3/
https://cupofexcellence.org/directory/90-27-2/
https://cupofexcellence.org/directory/90-14-6/
https://cupofexcellence.org/directory/90-14-5/
https://cupofexcellence.org/directory/89-55-3/
https://cupofexcellence.org/directory/89-18-4/
https://cupofexcellence.org/directory/89-14-2/
https://cupofexcellence.org/directory/88-66-4/
https://cupofexcellence.org/directory/88-59-11/
https://cupofexcellence.org/directory/88-57-3/
https://cupofexcellence.org/directory/88-55-3/
https://cupofexcellence.org/directory/88-5-7/
https://cupofexcellence.org/directory/88-23-7/
https://cupofexcellence.org/directory/88-90/
https://cupofexcellence.org/directory/87-95-4/
https://cupofexcellence.org/directory/87-86-7/
https://cupofexcellence.org/directory/87-82-7/
https://cupofexcellence.org/directory/87-77-6/
https://cupofexcellence.org/directory/87-73-6/
https://cupofe

https://cupofexcellence.org/directory/87-63-9/
https://cupofexcellence.org/directory/87-61-5/
https://cupofexcellence.org/directory/87-57-8/
https://cupofexcellence.org/directory/87-5-3/
https://cupofexcellence.org/directory/87-5-5/
https://cupofexcellence.org/directory/87-46-10/
https://cupofexcellence.org/directory/87-39-4/
https://cupofexcellence.org/directory/87-21-7/
https://cupofexcellence.org/directory/87-16-6/
https://cupofexcellence.org/directory/90-16/
https://cupofexcellence.org/directory/90-16-2/
https://cupofexcellence.org/directory/90-14-3/
https://cupofexcellence.org/directory/90-14-4/
https://cupofexcellence.org/directory/90-03-12/
https://cupofexcellence.org/directory/90-03-13/
https://cupofexcellence.org/directory/89-98/
https://cupofexcellence.org/directory/89-72-4/
https://cupofexcellence.org/directory/89-42-4/
https://cupofexcellence.org/directory/89-41-6/
https://cupofexcellence.org/directory/89-25-5/
https://cupofexcellence.org/directory/88-83-3/
https://cupofexc

https://cupofexcellence.org/directory/87-43-8/
https://cupofexcellence.org/directory/87-34-5/
https://cupofexcellence.org/directory/87-16-8/
https://cupofexcellence.org/directory/90-53-4/
https://cupofexcellence.org/directory/90-53-5/
https://cupofexcellence.org/directory/90-06-5/
https://cupofexcellence.org/directory/90-06-6/
https://cupofexcellence.org/directory/90-03-10/
https://cupofexcellence.org/directory/90-03-11/
https://cupofexcellence.org/directory/89-56-6/
https://cupofexcellence.org/directory/89-53-4/
https://cupofexcellence.org/directory/89-44-4/
https://cupofexcellence.org/directory/89-31-5/
https://cupofexcellence.org/directory/88-94-5/
https://cupofexcellence.org/directory/88-92-3/
https://cupofexcellence.org/directory/88-69-2/
https://cupofexcellence.org/directory/87-74-7/
https://cupofexcellence.org/directory/87-71-5/
https://cupofexcellence.org/directory/87-65-2/
https://cupofexcellence.org/directory/87-65-3/
https://cupofexcellence.org/directory/87-62-2/
https://cup

https://cupofexcellence.org/directory/88-87-2/
https://cupofexcellence.org/directory/88-5/
https://cupofexcellence.org/directory/87-83-4/
https://cupofexcellence.org/directory/87-8-2/
https://cupofexcellence.org/directory/87-77-3/
https://cupofexcellence.org/directory/87-73-4/
https://cupofexcellence.org/directory/87-63-8/
https://cupofexcellence.org/directory/87-57-7/
https://cupofexcellence.org/directory/87-43-6/
https://cupofexcellence.org/directory/87-33-6/
https://cupofexcellence.org/directory/87-23-4/
https://cupofexcellence.org/directory/87-23-5/
https://cupofexcellence.org/directory/87-2-2/
https://cupofexcellence.org/directory/87-17-14/
https://cupofexcellence.org/directory/87-17-15/
https://cupofexcellence.org/directory/87-07-7/
https://cupofexcellence.org/directory/86-67-10/
https://cupofexcellence.org/directory/86-57-13/
https://cupofexcellence.org/directory/86-53-11/
https://cupofexcellence.org/directory/86-4/
https://cupofexcellence.org/directory/86-17-10/
https://cupofex

...

https://cupofexcellence.org/directory/85-96/
https://cupofexcellence.org/directory/85-67/
https://cupofexcellence.org/directory/85-65-2/
https://cupofexcellence.org/directory/85-50-6/
https://cupofexcellence.org/directory/85-24-2/
https://cupofexcellence.org/directory/85-09-2/
https://cupofexcellence.org/directory/84-98-5/
https://cupofexcellence.org/directory/84-83-7/
https://cupofexcellence.org/directory/84-80-2/
https://cupofexcellence.org/directory/84-78/
https://cupofexcellence.org/directory/84-76-3/
https://cupofexcellence.org/directory/84-50-5/
https://cupofexcellence.org/directory/84-46-3/
https://cupofexcellence.org/directory/84-31-3/
https://cupofexcellence.org/directory/91-41/
https://cupofexcellence.org/directory/91-36/
https://cupofexcellence.org/directory/90-36/
https://cupofexcellence.org/directory/90-14/
https://cupofexcellence.org/directory/89-86/
https://cupofexcellence.org/directory/89-73/
https://cupofexcellence.org/directory/89-59/
https://cupofexcellence.org/direc

https://cupofexcellence.org/directory/85-98-4/
https://cupofexcellence.org/directory/85-48/
https://cupofexcellence.org/directory/85-36-2/
https://cupofexcellence.org/directory/85-30/
https://cupofexcellence.org/directory/85-30-2/
https://cupofexcellence.org/directory/85-16-2/
https://cupofexcellence.org/directory/85-14/
https://cupofexcellence.org/directory/85-00-2/
https://cupofexcellence.org/directory/84-93-2/
https://cupofexcellence.org/directory/84-91-2/
https://cupofexcellence.org/directory/84-82-2/
https://cupofexcellence.org/directory/84-80/
https://cupofexcellence.org/directory/84-77-3/
https://cupofexcellence.org/directory/84-75-6/
https://cupofexcellence.org/directory/84-68-2/
https://cupofexcellence.org/directory/84-68-3/
https://cupofexcellence.org/directory/84-57/
https://cupofexcellence.org/directory/84-57-2/
https://cupofexcellence.org/directory/84-48-2/
https://cupofexcellence.org/directory/84-39-2/
https://cupofexcellence.org/directory/84-36-2/
https://cupofexcellence

https://cupofexcellence.org/directory/86-05/
https://cupofexcellence.org/directory/85-87-2/
https://cupofexcellence.org/directory/85-70/
https://cupofexcellence.org/directory/85-43/
https://cupofexcellence.org/directory/85-33/
https://cupofexcellence.org/directory/84-63/
https://cupofexcellence.org/directory/84-60-2/
https://cupofexcellence.org/directory/84-02/
https://cupofexcellence.org/directory/91-40/
https://cupofexcellence.org/directory/91-27/
https://cupofexcellence.org/directory/90-90/
https://cupofexcellence.org/directory/90-50/
https://cupofexcellence.org/directory/89-92/
https://cupofexcellence.org/directory/89-46/
https://cupofexcellence.org/directory/89-15/
https://cupofexcellence.org/directory/88-79-2/
https://cupofexcellence.org/directory/88-44-2/
https://cupofexcellence.org/directory/88-07/
https://cupofexcellence.org/directory/85-95-2/
https://cupofexcellence.org/directory/85-89-3/
https://cupofexcellence.org/directory/85-87/
https://cupofexcellence.org/directory/85-73

https://cupofexcellence.org/directory/83-82/
https://cupofexcellence.org/directory/83-19/
https://cupofexcellence.org/directory/83-14/
https://cupofexcellence.org/directory/82-95/
https://cupofexcellence.org/directory/82-83/
https://cupofexcellence.org/directory/82-83-2/
https://cupofexcellence.org/directory/82-64-2/
https://cupofexcellence.org/directory/82-64/
https://cupofexcellence.org/directory/82-55/
https://cupofexcellence.org/directory/82-50/
https://cupofexcellence.org/directory/82-32/
https://cupofexcellence.org/directory/82-14/
https://cupofexcellence.org/directory/81-79/
https://cupofexcellence.org/directory/81-74/
https://cupofexcellence.org/directory/81-64/
https://cupofexcellence.org/directory/81-33/
https://cupofexcellence.org/directory/81-19/
https://cupofexcellence.org/directory/81-09/
https://cupofexcellence.org/directory/80-64/
https://cupofexcellence.org/directory/80-62/
https://cupofexcellence.org/directory/95-35/
https://cupofexcellence.org/directory/93-72/
https:

https://cupofexcellence.org/directory/1-00-43/
https://cupofexcellence.org/directory/1-00-44/
https://cupofexcellence.org/directory/1-00-45/
https://cupofexcellence.org/directory/1-00-46/
https://cupofexcellence.org/directory/1-00-11/
https://cupofexcellence.org/directory/1-00-12/
https://cupofexcellence.org/directory/1-00-13/
https://cupofexcellence.org/directory/1-00-14/
https://cupofexcellence.org/directory/1-00-15/
https://cupofexcellence.org/directory/1-00-16/
https://cupofexcellence.org/directory/1-00-17/
https://cupofexcellence.org/directory/1-00-18/
https://cupofexcellence.org/directory/1-00-19/
https://cupofexcellence.org/directory/1-00-20/
https://cupofexcellence.org/directory/1-00-21/
https://cupofexcellence.org/directory/1-00-22/
https://cupofexcellence.org/directory/1-00-23/
https://cupofexcellence.org/directory/1-00-24/
https://cupofexcellence.org/directory/1-00-25/
https://cupofexcellence.org/directory/1-00-26/
https://cupofexcellence.org/directory/1-00-27/
https://cupof